In [1]:
import subprocess
import os
import traceback
import json

In [16]:
def convertAllVideosToFrames(via_json_dir, video_dir, video_frame_dir):
    frame_filename_list = []

    for _, _, filenames in os.walk(via_json_dir):
        # sort ascending
        filenames_sorted = sorted(filenames)

        for f in filenames_sorted:
            if os.path.splitext(f)[1] == ".json":
                
                if "_2" in f:
                    video_filename, _ = f.split("_2")
                else:
                    video_filename = os.path.splitext(f)[0]

                video_filename += ".mp4"
                frame_filename = os.path.splitext(f)[0] + "_%05d.jpg"

                frame_filename_list.append((os.path.join(video_dir, video_filename), os.path.join(video_frame_dir, frame_filename)))

    video_with_error = []

    for video_path, video_frame_path in frame_filename_list: 
        try: 
            convertVideoToFrame(video_path, video_frame_path)
            print()

        except:
            trace_error = traceback.format_exc()

            print(trace_error)

            print("xxxxxxxxxxx WARNING! xxxxxxxxxxx")
            print(f"xxxxxxx video path = {video_path} xxxxxxx")
            print(f"xxxxxxx video frame path = {video_frame_path} xxxxxxx")
            print()
            
            video_with_error.append((video_path, video_frame_path, trace_error))

            pass

    if video_with_error != []:
        print("------------ List of Videos that Have Errors When Converting ------------")
        print("     format: (video path, video frame path)")
        for video_path, video_frame_path, trace_error in video_with_error:
            print((video_path, video_frame_path))
            print(trace_error)
            print()


def printStdOutput(std_output):
    output_split = std_output.split('\n')
    for output in output_split:
        print(output)


def convertVideoToFrame(video_path, video_frame_path):
    print(f"Converting video = {video_path}")
    print(f" To frame filenames = {video_frame_path}")

    # https://janakiev.com/blog/python-shell-commands/

    process = subprocess.Popen(['ffmpeg',  '-i', video_path, 
                                            '-vf', 'fps=10', 
                                            '-start_number', '0', 
                                            video_frame_path],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     universal_newlines=True)

    stdout, stderr = process.communicate()
    printStdOutput(stdout)
    printStdOutput(stderr)


In [17]:
convertAllVideosToFrames("./via_annotations/", "./videos/", "./frames/")

Converting video = ./videos/20200805_OneSharkSUPFollowsKids.mp4
 To frame filenames = ./frames/20200805_OneSharkSUPFollowsKids_%05d.jpg

ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 12.0.1
  configuration: --prefix=/Users/lunay_yuki/opt/anaconda3/envs/cs153 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1645955481026/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.

In [15]:
convertVideoToFrame("./videos/20200805_OneSharkSUPSurfers.mp4", "./frames/20200805_OneSharkSUPSurfers_%05d.jpg")


ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 12.0.1
  configuration: --prefix=/Users/lunay_yuki/opt/anaconda3/envs/cs153 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1645955481026/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './vid

In [1]:
test = '20210121_SUPNexttoShark_2.json'
print(test.split("_2"))

['20210121_SUPNexttoShark', '.json']


In [4]:
os.path.join(".", '20210121_SUPNexttoShark')

'./20210121_SUPNexttoShark'

In [9]:
def generatetVidToFileIdMap(via_json_dir, map_json_save_path):
    video_file_id_map = {"filenames": [], "id_map": {}}

    for _, _, filenames in os.walk(via_json_dir):
        # sort ascending
        filenames_sorted = sorted(filenames)

        via_json_files_sorted = [f for f in filenames_sorted if os.path.splitext(f)[1] == ".json"]

        for i in range(len(via_json_files_sorted)):
            f = via_json_files_sorted[i]
            
            if "_2." in f:
                video_filename, _ = f.split("_2")
            else:
                video_filename = os.path.splitext(f)[0]

            if video_filename not in video_file_id_map["filenames"]:
                video_file_id_map["filenames"].append(video_filename)

            if video_filename not in video_file_id_map["id_map"]:
                video_file_id_map["id_map"][video_filename] = [i]
            else:
                video_file_id_map["id_map"][video_filename].append(i)


    with open(map_json_save_path, "w") as f:
        json.dump(video_file_id_map, f)

In [10]:
generatetVidToFileIdMap("./via_annotations/", "./video_file_id_map.json")